In [ ]:
pip install portkey_ai

In [ ]:
from google.colab import userdata

In [ ]:
from portkey_ai import Portkey

portkey = Portkey(
  api_key = userdata.get('PORT_KEY')
)

response = portkey.chat.completions.create(
    model = "@first-integrati-db9427/gemini-2.0-flash-lite",
    messages = [
      {"role": "system", "content": "You are a helpful assistant."},
      {"role": "user", "content": "What is Portkey"}
    ],
    MAX_TOKENS = 512
)

print(response.choices[0].message.content)


In [ ]:
import pandas as pd

# Read CSV files (adjust file paths to where your CSVs are saved)
df_14_15 = pd.read_csv("df_14_15.csv") #change PATH is required


In [ ]:
pd.reset_option("display.max_colwidth")
pd.reset_option("display.max_columns")
df_14_15.head(10)

In [ ]:
from portkey_ai import Portkey
import time

modern_titles = []
modern_texts = []

delay_between_requests = 1  # seconds

# Loop through only the first 10 rows
for idx, row in df_14_15.head(10).iterrows():
    # Prompt for title
    title_prompt = (
        "Translate the following historical book title from the 1400–1500s into modern English. "
        "Do not add any explanation, only return the translated title:\n\n"
        f"'{row['Title']}'"
    )

    # Prompt for full text
    text_prompt = (
        "Translate the following passage from Early Modern English (1400–1500s) into clear, modern English prose. "
        "Retain the original meaning and tone as much as possible. Only return the translated passage:\n\n"
        f"{row['Text']}"
    )

    # Translate title
    try:
        response_title = portkey.chat.completions.create(
            messages=[{"role": "user", "content": title_prompt}],
            model="@first-integrati-db9427/gemini-2.0-flash-lite",
            max_completion_tokens=1000
        )
        modern_titles.append(response_title.choices[0].message.content.strip())
    except Exception as e:
        modern_titles.append(f"[Error in title: {str(e)}]")

    time.sleep(delay_between_requests)

    # Translate text
    try:
        response_text = portkey.chat.completions.create(
            messages=[{"role": "user", "content": text_prompt}],
            model="@first-integrati-db9427/gemini-2.0-flash-lite",
            max_completion_tokens=1000
        )
        modern_texts.append(response_text.choices[0].message.content.strip())
    except Exception as e:
        modern_texts.append(f"[Error in text: {str(e)}]")

    print(f"Processed row {idx + 1}/10")
    time.sleep(delay_between_requests)


first10 = df_14_15.head(10).index
df_14_15.loc[first10, "modern_title"] = modern_titles
df_14_15.loc[first10, "modern_text"]  = modern_texts

Personal Project Code

In [ ]:
from portkey_ai import Portkey
import time

modern_titles = []
modern_texts = []

delay_between_requests = 1  # seconds

# Loop through only the first 10 rows
for idx, row in df_14_15.head(10).iterrows():
    # Prompt for title
    title_prompt = (
        "Translate the following historical book title from the 1400–1500s into modern English. "
        "Do not add any explanation, only return the translated title:\n\n"
        f"'{row['Title']}'"
    )

    # Prompt for full text
    text_prompt = (
        "Translate the following passage from Early Modern English (1400–1500s) into clear, modern English prose. "
        "Retain the original meaning and tone as much as possible. Only return the translated passage:\n\n"
        f"{row['Text']}"
    )

    # Translate title
    try:
        response_title = portkey.chat.completions.create(
            messages=[{"role": "user", "content": title_prompt}],
            model="@first-integrati-db9427/gemini-2.0-flash-lite",
            max_completion_tokens=1000
        )
        modern_titles.append(response_title.choices[0].message.content.strip())
    except Exception as e:
        modern_titles.append(f"[Error in title: {str(e)}]")

    time.sleep(delay_between_requests)

    # Translate text
    try:
        response_text = portkey.chat.completions.create(
            messages=[{"role": "user", "content": text_prompt}],
            model="@first-integrati-db9427/gemini-2.0-flash-lite",
            max_completion_tokens=1000
        )
        modern_texts.append(response_text.choices[0].message.content.strip())
    except Exception as e:
        modern_texts.append(f"[Error in text: {str(e)}]")

    print(f"Processed row {idx + 1}/10")
    time.sleep(delay_between_requests)


first10 = df_14_15.head(10).index
df_14_15.loc[first10, "modern_title"] = modern_titles
df_14_15.loc[first10, "modern_text"]  = modern_texts

In [ ]:
first10 = df_14_15.head(10).index
df_14_15.loc[first10, "modern_title"] = modern_titles
df_14_15.loc[first10, "modern_text"]  = modern_texts

In [ ]:
# Show full column content (no truncation)
pd.set_option("display.max_colwidth", None)

# Show all columns if you have many
pd.set_option("display.max_columns", None)

df_14_15.head(5)


In [ ]:
#Save df as a new csv
df_14_15.to_csv("df_14_15_updated.csv", index=False)

# For Visualisation

In [ ]:
from collections import Counter
import matplotlib.pyplot as plt
import re
import nltk

# Download NLTK stopwords if not already
nltk.download("stopwords")
from nltk.corpus import stopwords
stop_words = set(stopwords.words("english"))

# Combine modern_title and modern_text into one big string
all_text = " ".join(
    df_14_15["modern_title"].dropna().astype(str) + " " +
    df_14_15["modern_text"].dropna().astype(str)
)

# Tokenize: keep only words (letters/numbers)
tokens = re.findall(r"\b\w+\b", all_text.lower())

# Remove stopwords
tokens = [t for t in tokens if t not in stop_words]

# Count most common words
word_counts = Counter(tokens).most_common(20)

# Plot
words, counts = zip(*word_counts)
plt.figure(figsize=(10,6))
plt.bar(words, counts)
plt.xticks(rotation=45)
plt.title("Top 20 Most Frequent Words (Translated Columns)")
plt.ylabel("Frequency")
plt.show()


# RAG with this data

In [ ]:
##For RAG
!pip install pandas faiss-cpu sentence-transformers portkey-ai streamlit

In [ ]:
#To Build Index
import json
import pandas as pd
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

CSV_PATH = "df_14_15_updated.csv" #change Path to where your CSV is
META_PATH  = "eebo_1415_meta.json"
INDEX_PATH = "eebo_1415_allmini.cosine.faiss"

df = pd.read_csv(CSV_PATH, low_memory=False)
df = df[["modern_title", "modern_text"]].dropna(subset=["modern_text"]).reset_index(drop=True)

def chunk_text(t, size=800, overlap=100):
    t = str(t)
    if len(t) <= size:
        return [t]
    chunks, start = [], 0
    while start < len(t):
        end = start + size
        chunks.append(t[start:end])
        if end >= len(t): break
        start = end - overlap
    return chunks

docs, meta = [], []
for i, row in df.iterrows():
    title = (row["modern_title"] or "")
    text  = str(row["modern_text"])
    for j, ch in enumerate(chunk_text(text)):
        docs.append((title + "\n\n" + ch).strip())
        meta.append({"row_id": int(i), "chunk_id": int(j), "title": title})

model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
embs = model.encode(docs, convert_to_numpy=True, normalize_embeddings=True)

dim = embs.shape[1]
index = faiss.IndexFlatIP(dim)  # cosine if vectors are normalized
index.add(embs)

faiss.write_index(index, INDEX_PATH)
with open(META_PATH, "w") as f:
    json.dump(meta, f)

print(f"Built index with {len(docs)} chunks → {INDEX_PATH}")


In [ ]:
#To chat using LLM API's from Portkey
import os, json
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss
from portkey_ai import Portkey

#INDEX_PATH = "eebo_1415.faiss"
INDEX_PATH = "eebo_1415_allmini.cosine.faiss"
META_PATH  = "eebo_1415_meta.json"
CSV_PATH   = "df_14_15_updated.csv" #add in your PATH

# --- Load retrieval artifacts ---
index = faiss.read_index(INDEX_PATH)
with open(META_PATH) as f:
    META = json.load(f)
df = pd.read_csv(CSV_PATH, low_memory=False)[["modern_title", "modern_text"]]

# Embedder (must match build_index.py)
embedder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

def retrieve(query, k=4, max_ctx_chars=2000):
    q = embedder.encode([query], normalize_embeddings=True).astype(np.float32)
    D, I = index.search(q, k)
    hits = []
    for idx, score in zip(I[0], D[0]):
        m = META[idx]
        row = df.iloc[m["row_id"]]
        context = (str(row["modern_title"]) + "\n\n" + str(row["modern_text"])).strip()
        hits.append({
            "score": float(score),
            "row_id": m["row_id"],
            "chunk_id": m["chunk_id"],
            "title": m["title"],
            "context": context[:max_ctx_chars]
        })
    return hits

# --- Portkey client (your format) ---
portkey = Portkey(
  api_key = userdata.get('PORT_KEY')
)

MODEL = "@first-integrati-db9427/gemini-2.0-flash-lite"

def call_llm_portkey(system_prompt, user_prompt):
    response = portkey.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        max_tokens=512,
    )
    return response.choices[0].message["content"]

def answer(query):
    contexts = retrieve(query, k=4)
    ctx_block = "\n\n---\n".join(
        [f"[{i+1}] Title: {c['title']}\n{c['context']}" for i, c in enumerate(contexts)]
    )
    cites = "\n".join([f"- [{i+1}] row_id={c['row_id']} chunk={c['chunk_id']} title={c['title']}"
                       for i, c in enumerate(contexts)])
    system = (
        "You are a helpful assistant. Use ONLY the provided CONTEXT to answer.\n"
        "If insufficient, say you don't know."
    )
    user = f"QUESTION: {query}\n\nCONTEXT:\n{ctx_block}\n\nReturn a concise answer"
    out = call_llm_portkey(system, user)
    return out, cites

if __name__ == "__main__":
    print("RAG chat ready (Portkey). Type a question (Ctrl+C to quit).")
    while True:
        try:
            q = input("\nYou: ").strip()
            if not q:
                continue
            ans, src = answer(q)
            print("\nAssistant:", ans)
            print("\nSources:\n", src)
        except KeyboardInterrupt:
            print("\nBye!")
            break


Complaint Data

In [ ]:
import pandas as pd

def narrative(r):
    date = f"On {r['incident_date']}" if r['incident_date'] != "Unlisted" else "On an unknown date"
    if r['first_name'] != "Unlisted" and r['last_name'] != "Unlisted":
        on = f", {r['complaint_id']} was filed against officer {r['first_name']}, {r['last_name']}"
    else:
        on = ", against an unknown officer"
    where= f" at a(n) {r['location_type']}." if r['location_type'] != "Unlisted" else " at an unknown location."
    reason =  f" for the reason: \"{r['contact_reason']}\"."
    alleg = f" The allegation was classified with the FADO type of {r['fado_type']} with a specific allegation of {r['allegation_cat']}."
    outcome = f" The contact outcome was {r['contact_outcome']}."
    ccrb_disposition = f"The ccrb_disposition is \"{r['ccrb_disposition']}\"." if r['ccrb_disposition'] != " Complainant Unavailable" else " The ccrb_disposition is unavailable."
    penalty = f" The penalty received is {r['penalty_rec']}." if r['penalty_rec'] != "Not Applicable" else " There was no penalty received."
    status_cat = f" The complaint status is {r['status_cat']} as of 4/1/2021." if r['status_cat'] != "Unlisted" else " The complaint status is unknown"
    overview = date + on + where + reason + alleg + outcome + ccrb_disposition + penalty + status_cat;

    officer_race = f"The officer race is {r['officer_race']}." if r['officer_race'] != "Unlisted" else "The officer race is unknown."
    officer_sex = f" The officer sex is {r['officer_gender']}." if r['officer_gender'] != "Unlisted" else " The officer sex is unknown."
    days_on_force = f" The officer was on force for {r['days_on_force']} days when this dataset was last recorded." if r['days_on_force'] != "Unlisted" else " The officer was on force for an unknown number of days when this dataset was last recorded."
    officer_incident_rank = f" The officer rank at the time of the incident was {r['officer_incident_rank']}." if r['officer_incident_rank'] != "Unlisted" else " The officer rank during the incident is unknown."
    officer_current_rank = f" The officer current rank is {r['officer_current_rank']}." if r['officer_current_rank'] != "Unlisted" else " The officer current rank is unknown."
    officer_statistics = officer_race + officer_sex + days_on_force + officer_incident_rank + officer_current_rank;

    impacted_race = f"The race of the victim / alleged victim is {r['impacted_race']}" if r['impacted_race'] != "Unlisted" else "The race of the victim / alleged victim is unknown."
    impacted_gender = f" The gender of the victim / alleged victim is {r['impacted_gender']}" if r['impacted_gender'] != "Unlisted" else " The gender of the victim / alleged victim is unknown."
    impacted_statistics = impacted_race + impacted_gender;

    summary = (
    f"{overview}\n\n"
    f"{officer_statistics}\n\n"
    f"{impacted_statistics}"
    )
    return summary


pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)

df = pd.read_csv("complaintclean.csv")
df = df.head()
summary = df.apply(narrative, axis=1)
summary
#print(df.head())


In [ ]:
# Corrected code to assign the summary to a new column
df['summary'] = summary

# Display the first few rows with the new 'summary' column
display(df.head())

In [ ]:
# Read CSV files (adjust file paths to where your CSVs are saved)
df.to_csv("complaintclean_narrative.csv", index=False)

In [ ]:
##For RAG
!pip install pandas faiss-cpu sentence-transformers portkey-ai streamlit

In [ ]:
#To Build Index
import json
import pandas as pd
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

CSV_PATH = "complaintclean_narrative.csv" #change Path to where your CSV is
META_PATH  = "eebo_cc_meta.json"
INDEX_PATH = "eebo_cc_allmini.cosine.faiss"

df = pd.read_csv(CSV_PATH, low_memory=False)
df = df[["complaint_id", "summary"]].dropna(subset=["summary"]).reset_index(drop=True)

def chunk_text(t, size=800, overlap=100):
    t = str(t)
    if len(t) <= size:
        return [t]
    chunks, start = [], 0
    while start < len(t):
        end = start + size
        chunks.append(t[start:end])
        if end >= len(t): break
        start = end - overlap
    return chunks

docs, meta = [], []
for i, row in df.iterrows():
    title = str((row["complaint_id"] or ""))
    text  = str(row["summary"])
    for j, ch in enumerate(chunk_text(text, size=200)):
        docs.append((title + "\n\n" + ch).strip())
        meta.append({"row_id": int(i), "chunk_id": int(j), "title": title})

model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
embs = model.encode(docs, convert_to_numpy=True, normalize_embeddings=True)

dim = embs.shape[1]
index = faiss.IndexFlatIP(dim)  # cosine if vectors are normalized
index.add(embs)

faiss.write_index(index, INDEX_PATH)
with open(META_PATH, "w") as f:
    json.dump(meta, f)

print(f"Built index with {len(docs)} chunks → {INDEX_PATH}")


In [ ]:
#To chat using LLM API's from Portkey
import os, json
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss
from portkey_ai import Portkey
from google.colab import userdata

#INDEX_PATH = "eebo_1415.faiss"
INDEX_PATH = "eebo_cc_allmini.cosine.faiss"
META_PATH  = "eebo_cc_meta.json"
CSV_PATH   = "complaintclean_narrative.csv" #add in your PATH

# --- Load retrieval artifacts ---
index = faiss.read_index(INDEX_PATH)
with open(META_PATH) as f:
    META = json.load(f)
df = pd.read_csv(CSV_PATH, low_memory=False)[["complaint_id", "summary"]]

# Embedder (must match build_index.py)
embedder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

def retrieve(query, k=4, max_ctx_chars=2000):
    q = embedder.encode([query], normalize_embeddings=True).astype(np.float32)
    D, I = index.search(q, k)
    hits = []
    for idx, score in zip(I[0], D[0]):
        m = META[idx]
        row = df.iloc[m["row_id"]]
        context = (str(row["complaint_id"]) + "\n\n" + str(row["summary"])).strip()
        hits.append({
            "score": float(score),
            "row_id": m["row_id"],
            "chunk_id": m["chunk_id"],
            "title": m["title"],
            "context": context[:max_ctx_chars]
        })
    return hits

# --- Portkey client (your format) ---
portkey = Portkey(
  api_key = userdata.get('PORT_KEY')
)

MODEL = "@first-integrati-db9427/gemini-2.0-flash-lite"

def call_llm_portkey(system_prompt, user_prompt):
    response = portkey.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        max_tokens=512,
    )
    return response.choices[0].message["content"]

def answer(query):
    contexts = retrieve(query, k=2)
    ctx_block = "\n\n---\n".join(
        [f"[{i+1}] Title: {c['title']}\n{c['context']}" for i, c in enumerate(contexts)]
    )
    cites = "\n".join([f"- [{i+1}] row_id={c['row_id']} chunk={c['chunk_id']} title={c['title']}"
                       for i, c in enumerate(contexts)])
    system = (
        "You are a helpful assistant. Use the provided CONTEXT to answer.\n"
        "but if you need to, use verified external knowledge and make a disclaimer to let"
        "the user know that you're using external data"
        "Don't be afraid to summarize all of the data"
        "If insufficient, say you don't know."
    )
    user = f"QUESTION: {query}\n\nCONTEXT:\n{ctx_block}\n\nReturn a concise answer"
    out = call_llm_portkey(system, user)
    return out, cites

if __name__ == "__main__":
    print("RAG chat ready (Portkey). Type a question (Ctrl+C to quit).")
    while True:
        try:
            q = input("\nYou: ").strip()
            if not q:
                continue
            ans, src = answer(q)
            print("\nAssistant:", ans)
            print("\nSources:\n", src)
        except KeyboardInterrupt:
            print("\nBye!")
            break